# E4729 Lecture 6-02: Tick Data - Merging Quote and Trade Data for Replay

In [1]:
# data source and spec
# Files: ftp://ftp.nyxdata.com/Historical%20Data%20Samples/Daily%20TAQ%20Sample%202017/
# Spec: http://www.nyxdata.com/doc/247075

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as md
import numpy as np
import pandas as pd
import time
import datetime

matplotlib.rcParams[ 'figure.figsize' ] = ( 14, 6 )

def show_time( label_string ):
    ts = time.time()
    st = datetime.datetime.fromtimestamp( ts ).strftime( '%Y-%m-%d %H:%M:%S:%f' )
    print( label_string + ' : ' + st )

In [3]:
quote_file = 'EQY_US_ALL_NBBO_20180117.gz'
trade_file = 'EQY_US_ALL_TRADE_20180117.gz'

file_date = '20180117'

In [ ]:
# load quotes
show_time( 'load quotes' )

quotes = pd.read_csv( quote_file, sep = '|', 
                      usecols = [ 'Time', 'Symbol', 'Bid_Price', 'Bid_Size', 
                                  'Offer_Price', 'Offer_Size', 'Quote_Condition', 'Exchange'],
                      error_bad_lines = False,
                      dtype = { 'Symbol' : object, 'Bid_Price' : float, 'Bid_Size' : float,
                                'Offer_Price' : float, 'Offer_Size' : float, 'Quote_Condition' : object, 
                                'Exchange' : object },
                      #nrows=10000
                    )
show_time( 'load quotes done' )

load quotes : 2019-03-28 22:58:51:092186


In [5]:
# add an index, converting to appropriate time and  correcting for the missing date.

# first strip the last record with "END" in it to avoid errors on conversion

show_time( 'index quotes' )
quotes = quotes.iloc[:-1]
format = '%Y%m%d%H%M%S%f'
quotes[ 'Time' ] = file_date + quotes[ 'Time' ].astype( str )
times = pd.to_datetime( quotes[ 'Time' ], format = format )
del quotes[ 'Time' ]
quotes.index = times
show_time( 'index quotes done' )

index quotes : 2019-03-28 22:57:44:496303
index quotes done : 2019-03-28 22:57:44:576756


In [6]:
# load trades
show_time( "load trades")
trades = pd.read_csv( trade_file, sep = '|',  
                      usecols = [ 'Time', 'Symbol', 'Trade Price', 'Trade Volume', 'Sale Condition' ],
                      error_bad_lines = False,
                      dtype = { 'Symbol' : object, 'Trade Price' : float, 'Trade Volume' : float,
                                'Sale Condition' : object },
                      #nrows=10000
                    )
show_time( "load trades done" )
# cut the last record with END
trades = trades.iloc[:-1]
format = '%Y%m%d%H%M%S%f'
trades[ 'Time' ] = file_date + trades[ 'Time' ].astype( str )
times = pd.to_datetime( trades[ 'Time' ], format = format )
del trades[ 'Time' ]
trades.index = times
show_time( "index trades done" )

load trades : 2019-03-28 22:57:45:520756
load trades done : 2019-03-28 22:57:45:544401
index trades done : 2019-03-28 22:57:45:614289


In [7]:
trades.head()

,Symbol,Sale Condition,Trade Volume,Trade Price
Time,,,,
2018-01-17 09:29:50.009602,A,FTI,25.0,71.54
2018-01-17 09:30:01.004090,A,O,14777.0,71.72
2018-01-17 09:30:02.512285,A,I,13.0,71.69
2018-01-17 09:30:02.573805,A,F I,20.0,71.56
2018-01-17 09:30:02.602246,A,F I,19.0,71.56


In [9]:
# uncomment to pre-filter on a ticker
ticker = 'AAPL'
quotes = quotes[ quotes[ 'Symbol' ] == ticker ]
trades = trades[ trades[ 'Symbol' ] == ticker ]

In [10]:
# now merge quote and tick data
show_time( 'start merge' )
taq = quotes.merge( trades, how = 'outer', on = 'Symbol', left_index = True, right_index = True )
show_time( 'end merge' )

start merge : 2019-03-28 22:58:18:374517
end merge : 2019-03-28 22:58:18:377767


In [11]:
len(taq)

0

In [12]:
taq.tail()

,Exchange,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Symbol,Sale Condition,Trade Volume,Trade Price
Time,,,,,,,,,,


In [9]:
# to save this to a file so we don't have to load and merge again
taq.to_csv( 'MY_TAQ_20180117.gz', compression = 'gzip' )

In [ ]:
# filtering the file by date / time
len(taq)
taq['2018-01-17 09:29':'2018-01-17 09:31']

In [11]:
len(ticker)

282110

In [13]:
fb = taq[taq['Symbol'] == 'FB']
msft = taq[taq['Symbol'] == 'MSFT']

In [ ]:
# now save our file
ticker.to_csv( 'VZ_20180117.gz', compression = 'gzip' )

In [14]:
len(fb)

425185

In [15]:
len(msft)

547756

In [16]:
fb.to_csv( 'FB_20180117.gz', compression = 'gzip' )
msft.to_csv( 'MSFT_20180117.gz', compression = 'gzip' )

In [30]:
def make_ticker_file( ticker, taq ):
    ticker_data = taq[ taq[ 'Symbol' ]  == ticker ]
    out_file = ticker + '_20180117.gz'
    print(out_file)
    ticker_data.to_csv( out_file, compression = 'gzip' )

In [31]:
make_ticker_file( 'AAPL', taq )

AAPL_20180117.gz
